In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf

from sklearn.preprocessing import MinMaxScaler 

from keras.utils.np_utils import to_categorical 
from keras.models import Sequential 
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, Dropout
from keras.optimizers import SGD 


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_data=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [4]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
x_train=train_data.drop(['label'], axis=1)
y_train=train_data['label']
x_test=test_data
scaler=MinMaxScaler()
y_train.value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [6]:
x_train=pd.DataFrame(scaler.fit_transform(x_train), columns=scaler.feature_names_in_)
x_test=pd.DataFrame(scaler.transform(x_test), columns=scaler.feature_names_in_)
x_train=x_train.values.reshape(-1,28,28,1)
x_test=x_test.values.reshape(-1,28,28,1)
y_train=to_categorical(y_train, num_classes=10)

In [7]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), kernel_initializer='he_uniform', padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(5,5), kernel_initializer='he_uniform', padding='Same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(5,5), kernel_initializer='he_uniform', padding='Same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(128, kernel_initializer='he_uniform', activation='relu'))
model.add(BatchNormalization())


model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                        

In [8]:
model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9,), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_split=0.1, batch_size=50, epochs=50, verbose=2)

predictions = model.predict(x_test)
n_predictions = np.argmax(predictions, axis=1)

Epoch 1/50
756/756 - 102s - loss: 0.1021 - accuracy: 0.9687 - val_loss: 0.0482 - val_accuracy: 0.9840 - 102s/epoch - 135ms/step
Epoch 2/50
756/756 - 99s - loss: 0.0300 - accuracy: 0.9909 - val_loss: 0.0343 - val_accuracy: 0.9883 - 99s/epoch - 131ms/step
Epoch 3/50
756/756 - 98s - loss: 0.0167 - accuracy: 0.9953 - val_loss: 0.0327 - val_accuracy: 0.9893 - 98s/epoch - 130ms/step
Epoch 4/50
756/756 - 99s - loss: 0.0080 - accuracy: 0.9983 - val_loss: 0.0310 - val_accuracy: 0.9900 - 99s/epoch - 131ms/step
Epoch 5/50
756/756 - 99s - loss: 0.0045 - accuracy: 0.9992 - val_loss: 0.0260 - val_accuracy: 0.9912 - 99s/epoch - 131ms/step
Epoch 6/50
756/756 - 99s - loss: 0.0030 - accuracy: 0.9996 - val_loss: 0.0235 - val_accuracy: 0.9919 - 99s/epoch - 131ms/step
Epoch 7/50
756/756 - 99s - loss: 0.0018 - accuracy: 0.9998 - val_loss: 0.0229 - val_accuracy: 0.9921 - 99s/epoch - 131ms/step
Epoch 8/50
756/756 - 99s - loss: 0.0011 - accuracy: 0.9999 - val_loss: 0.0228 - val_accuracy: 0.9919 - 99s/epoch - 1

In [9]:
pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')


,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [10]:
df = pd.DataFrame()
df['ImageId'] = range(1, 28001)
df['Label'] = n_predictions
df.to_csv('submission.csv', index=False)